In [2]:
"""ВИДІЛЕННЯ ВИЯВЛЕНИХ ТЗ НА ТЕСТОВІЙ ЗБІРЦІ"""

import os
import cv2
import numpy as np
import pywt
from ultralytics import YOLO

def apply_wavelet_centered(img, wavelet='db6', lambda_=0.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coeffs2 = pywt.dwt2(gray, wavelet)
    _, (H, V, D) = coeffs2
    W = np.sqrt(H ** 2 + V ** 2 + D ** 2)
    W = cv2.resize(W, (img.shape[1], img.shape[0]))
    W_centered = W - np.mean(W)
    img_float = img.astype(np.float32)
    W3 = np.stack([W_centered] * 3, axis=-1)
    result = img_float + lambda_ * W3
    return np.clip(result, 0, 255).astype(np.uint8)

def detect_on_test_images(model_path, test_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    model = YOLO(model_path)

    for filename in os.listdir(test_folder):
        if not filename.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue

        path = os.path.join(test_folder, filename)
        img = cv2.imread(path)
        if img is None:
            continue

        img_wavelet = apply_wavelet_centered(img, lambda_=0.5)
        results = model.predict(img_wavelet, conf=0.25, verbose=False)[0]

        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 1)

        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, img)


if __name__ == '__main__':
    detect_on_test_images(
        model_path='model.pt',
        test_folder='data/test/images',
        output_folder='res_im'
    )
